<a href="https://colab.research.google.com/github/ajinkyagorad/AskMe/blob/main/askMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.8 MB/s eta 0:00:00


In [11]:
pip install gTTS


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
# all imports
# https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3, file_path='input.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  #audio = AudioSegment.from_file(BytesIO(b))
  with open(file_path, 'wb') as f:
    f.write(b)
  
  

In [29]:
# Askme is a speech to text->text to chatGPT-> response to speech

import openai
import requests
import json
from gtts import gTTS
import IPython


# Replace with your own API keys
openai.api_key = <add openai keys with subscription>

def speech_to_text(audio_file):
    with open(audio_file, "rb") as f:
        transcript = openai.Audio.transcribe("whisper-1", f)
        return transcript['text']

def chatgpt_response(prompt):
    response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, max_tokens=50, n=1, stop=None, temperature=0.7)
    message = response.choices[0].text.strip()
    return message

def text_to_speech(text):
    tts = gTTS(text, lang="en", slow=False)
    tts.save("output.mp3")

def main():
    audio_file = "input.wav"  # Replace with the path to your input audio file

    record(10, audio_file)
    text = speech_to_text(audio_file)
    print("Speech to Text:", text)

    response = chatgpt_response(text)
    print("ChatGPT Response:", response)

    text_to_speech(response)
    print("Text to Speech: output.mp3")
    with open('output.mp3', "rb") as f:
      audio = f.read()
      IPython.display.clear_output()
      print()
      display(IPython.display.Audio(audio, rate=22050))
if __name__ == "__main__":
    main()
